In [2]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import dataset
import schedulers

import tensorflow as tf
import tensorflowjs as tfjs

assert tf.config.list_physical_devices('GPU')

In [3]:
BATCH_SIZE = 64

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)

def build_model(UNITS=500):
    inp = keras.Input(shape=(64,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, UNITS, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True), merge_mode='sum')(embed)
    layer = layers.add([layer, layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True), merge_mode='sum')(layer)])
    layer = layers.BatchNormalization()(layer)
    layer = layers.add([embed, layers.Dense(UNITS, activation='relu')(layer)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    model = keras.Model(inputs=inp, outputs=outputs)

    return model

model = build_model()

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 64, 500)      22000       input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 64, 500)      4004000     embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 64, 500)      4004000     bidirectional[0][0]              
______________________________________________________________________________________________

In [4]:

# masked version of accuracy and sce
def accuracy(real, pred):
    acc = tf.keras.metrics.sparse_categorical_accuracy(real, pred)

    mask = tf.cast(tf.math.logical_not(tf.math.equal(real, 0)), dtype=acc.dtype)
    acc *= mask

    return tf.reduce_sum(acc) / tf.reduce_sum(mask)

def sparse_categorical_crossentropy(y_true, y_pred, sample_weight=None):
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)

    mask = tf.cast(tf.math.logical_not(tf.math.equal(y_true, 0)), dtype=loss.dtype)
    loss *= mask

    return tf.reduce_sum(loss) / tf.reduce_sum(mask) 

def fit(train_validation, scheduler=None, verbose=1, lr=1e-4, epochs=1):
    train, valid = train_validation
    
    model.compile(loss=sparse_categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  metrics={'N': accuracy, 'D': accuracy, 'S': accuracy})
    callbacks = []
    if isinstance(scheduler, schedulers.CircularLearningRate):
        scheduler.set_dataset(train, BATCH_SIZE)
    if scheduler:
        callbacks.append(scheduler)    
    
    x  = train.normalized
    y  = {'N': train.niqqud, 'D': train.dagesh, 'S': train.sin }
    
    if valid is None:
        return model.fit(x, y, batch_size=BATCH_SIZE, epochs=epochs, verbose=verbose, callbacks=callbacks)
    
    vx = valid.normalized
    vy = {'N': valid.niqqud, 'D': valid.dagesh, 'S': valid.sin }
    
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=epochs, verbose=verbose, callbacks=callbacks)


MAXLEN = 64
def load_data(source, maxlen=MAXLEN, validation=0.1):
    filenames = [os.path.join('texts', f) for f in source]
    
    if validation == 0:
        corpus = dataset.read_corpus(filenames, maxlen)
        train = dataset.Data.concatenate(corpus)
        train.shuffle()
        return train, None
    
    train, valid = dataset.load_data(filenames, validation, maxlen=maxlen)
    return train, valid


def load_test_data(maxlen=MAXLEN):
    return dataset.load_file('test/ModernTestCorpus-HebrewWiki1-simple.txt', maxlen) 


In [19]:
data_mix = load_data(['poetry', 'rabanit', 'pre_modern'], validation=0)

In [20]:
data_modern = load_data(['modern'], validation=0)

In [21]:
model.load_weights('./checkpoints/uninit')
history = fit(data_mix, scheduler=schedulers.CircularLearningRate(30e-4, 80e-4, 1e-4), epochs=1)
model.save_weights('./checkpoints/mix')

2426/2426 [==============================] - 244s 101ms/step - loss: 0.2524 - N_loss: 0.1936 - D_loss: 0.0525 - S_loss: 0.0063 - N_accuracy: 0.9318 - D_accuracy: 0.9790 - S_accuracy: 0.99830s - loss: 0.2 - ETA: 0s - loss: 0.2525 - N_loss: 0.1936 - D_loss: 0.0525 - S_loss: 0.0063 - N_accuracy: 0.9317 - D_accuracy: 0.9790 - S_accuracy: 0.9


In [22]:
model.load_weights('./checkpoints/mix')
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(50e-4, 50e-4, 1e-5))
model.save_weights('./checkpoints/modern')

292/292 [==============================] - 29s 101ms/step - loss: 0.1444 - N_loss: 0.1078 - D_loss: 0.0346 - S_loss: 0.0021 - N_accuracy: 0.9636 - D_accuracy: 0.9869 - S_accuracy: 0.999517s - loss: 0.1814 - N_


In [23]:
model.load_weights('./checkpoints/modern')
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(5e-4, 15e-4, 1e-5), epochs=1)
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(5e-5, 1e-4, 1e-5), epochs=1)
model.save_weights('./checkpoints/modern_over')

292/292 [==============================] - 29s 101ms/step - loss: 0.0289 - N_loss: 0.0197 - D_loss: 0.0088 - S_loss: 4.0045e-04 - N_accuracy: 0.9936 - D_accuracy: 0.9968 - S_accuracy: 0.9999


In [24]:
model.load_weights('./checkpoints/modern_over')

model.compile()
model.save('modern.h5')
tfjs.converters.save_keras_model(model, '.')

In [ ]:
for i in range(50):
    model.load_weights('./checkpoints/pre_modern')
    p1 = np.exp(np.random.uniform(low=np.log(1e-5), high=np.log(1e-2)))
    p2 = np.exp(np.random.uniform(low=np.log(1e-4), high=np.log(1e-1)))
    p3 = np.exp(np.random.uniform(low=np.log(1e-5), high=np.log(1e-2)))
    print(p1, p2, p3, end=', ', sep=', ')
    history = fit(data_modern, scheduler=schedulers.CircularLearningRate(p1, p2, p3), verbose=0)
    print(history.history['val_N_accuracy'][0])

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=3)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['N', 'D', 'S'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [5]:
model.load_weights('./checkpoints/modern_over')
test = load_test_data()
x = test.normalized
y = {'N': test.niqqud, 'D': test.dagesh, 'S': test.sin }

model.compile(loss=sparse_categorical_crossentropy,
              metrics={'N': accuracy, 'D': accuracy, 'S': accuracy})

model.evaluate(x=x, y=y, batch_size=BATCH_SIZE)

10/10 [==============================] - 0s 37ms/step - loss: 0.2230 - N_loss: 0.1833 - D_loss: 0.0366 - S_loss: 0.0031 - N_accuracy: 0.9524 - D_accuracy: 0.9881 - S_accuracy: 0.9992


[0.2229769080877304,
 0.18329156935214996,
 0.03662189096212387,
 0.003063462907448411,
 0.9523879289627075,
 0.9881359338760376,
 0.9991891980171204]

In [7]:
model.load_weights('./checkpoints/modern_over')

def real_evaluation(data, s=slice(0, None), print_comparison=True):
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud[s], data.dagesh[s], data.sin[s]]
    actual = dataset.merge(data.text[s], ts=batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(data.text[s], ts=batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    total = []
    for i, (b, a, e) in enumerate(zip(batch, actual, expected)):
        res = ( (expected_niqqud[i][b > 16] == actual_niqqud[i][b > 16])
              & (expected_dagesh[i][b > 16] == actual_dagesh[i][b > 16])
              & (expected_sin[i][b > 16] == actual_sin[i][b > 16]) )
        total.extend(res)
        if print_comparison:
            print('מצוי: ', a)
            print('רצוי: ', e)
            print(f'{np.mean(res):.1%} ({len(res)-np.sum(res)} out of {len(res)})')
            print()
    print(f'{np.mean(total):.1%}')

real_evaluation(test, print_comparison=False)

92.5%
